# PruneMe
- Repo: https://github.com/arcee-ai/PruneMe
- Paper to cite: 
    @misc{gromov2024unreasonable,
      title={The Unreasonable Ineffectiveness of the Deeper Layers}, 
      author={Andrey Gromov and Kushal Tirumala and Hassan Shapourian and Paolo Glorioso and Daniel A. Roberts},
      year={2024},
      eprint={2403.17887},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
    }

- Repository PruneMe kopieren (git clone https://github.com/arcee-ai/PruneMe)
- Repository Mergekit kopieren (git clone https://github.com/arcee-ai/mergekit)
- Installation wie auf den Repos beschrieben durchführen (pip install -e .)

## Workflow PruneMe 
> ### Analyse des Ursprungs Modells 
> - Wechsel  in das Verzeichnis compute_block_similarity:
> - Ausführen des Tasks layer_similarity generiert eine CSV Datei welche im selben Verzeichnis abgelegt wird
> - Aus dieser Datei sucht man die Layer mit den geringsten Unterschied als Basis für den Merge Task heraus

> ### Erstellen des neuen  "Pruned" Modells
> - Wechsel in das Verzeichnis slice_with_mergekit
> - editieren der Datei slice.yaml mit den entsprechenden layern und modellnamen (siehe Beispiel unten)
> - ausführen von merge_me.py
> - das neue Modell wird im Verzeichnis "merged" abgelegt



## 0. Vorbereitungen
- Einmalig nach Notebook Start

In [1]:
conda activate PruneMe

(/home/thsch026/my-envs/PruneMe) 


### Generelle Pfade

In [22]:
export PMODELOUT="/home/thsch026/masterarbeit/models/generated/prune/pruneme/"
export PRESULTS="/home/thsch026/masterarbeit/Slim/results/PruneMe/"

(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 


## 1. Produktive Pruning Kompression
- Auswahl des Modells

In [4]:
export PRESULTS="/home/thsch026/masterarbeit/Slim/results/PruneMe/"
export PMODELOUT="/home/thsch026/masterarbeit/models/generated/prune/pruneme/"

### Llama-3-8B-Instruct-HF

#### 8 Layers extracted

In [5]:
export MODEL="/home/thsch026/masterarbeit/models/llama3/Meta-Llama-3-8B-Instruct-HF"
export LAYERSPRUNED="8"
export OUT="Meta-Llama-3-8B-Instruct-"$LAYERSPRUNED"_Ext"

#### 12 Layers extracted

In [7]:
export MODEL="meta-llama/Meta-Llama-3-8B-Instruct"
export LAYERSPRUNED="12"
export OUT="Meta-Llama-3-8B-Instruct-"$LAYERSPRUNED"_Ext"

### Llama-3-8B

#### 8 Layers Extracted

In [9]:
export MODEL="meta-llama/Meta-Llama-3-8B"
export LAYERSPRUNED="8"
export OUT="Meta-Llama-3-8B-"$LAYERSPRUNED"_Ext"

#### 12 Layers extracted

In [11]:
export MODEL="meta-llama/Meta-Llama-3-8B"
export LAYERSPRUNED="12"
export OUT="Meta-Llama-3-8B-"$LAYERSPRUNED"_Ext"

### Mistral-7B-Instruct

#### 8 Layers extracted

In [13]:
export MODEL="mistralai/Mistral-7B-Instruct-v0.2"
export LAYERSPRUNED="8"
export OUT="Mistral-7B-Instruct-v0.2-"$LAYERSPRUNED"_Ext"

#### 12 Layers extracted

In [15]:
export MODEL="mistralai/Mistral-7B-Instruct-v0.2"
export LAYERSPRUNED="12"
export OUT="Mistral-7B-Instruct-v0.2-"$LAYERSPRUNED"_Ext"

### Llama2-7B-chat

#### 8 Layers extracted

In [17]:
export MODEL="meta-llama/Llama-2-7b-chat-hf"
export LAYERSPRUNED="8"
export OUT="Llama-2-7b-chat-hf-"$LAYERSPRUNED"_Ext"

#### 12 Layers extracted

In [76]:
export MODEL="meta-llama/Llama-2-7b-chat-hf"
export LAYERSPRUNED="12"
export OUT="Llama-2-7b-chat-hf-"$LAYERSPRUNED"_Ext"

(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 


### Beipiel für den Inhalt von slice.yaml

In [ ]:
slices:
  - sources:
      - model: /home/thsch026/masterarbeit/models/llama3/Meta-Llama-3-8B-Instruct-HF
        layer_range: [0, 22]
  - sources:
      - model: /home/thsch026/masterarbeit/models/llama3/Meta-Llama-3-8B-Instruct-HF
        layer_range: [30,32]
            
merge_method: passthrough
dtype: bfloat16

###  2. Setzen der Directories

In [18]:
export PRUNEMODEL="$PMODELOUT$OUT"
export RESULTS="$PRESULTS$OUT"
echo $PRUNEMODEL

/home/thsch026/masterarbeit/models/generated/prune/pruneme/Llama-2-7b-chat-hf-8_Ext


### 3. Festlegen der GPU für die Analyse

In [73]:
export CUDA_VISIBLE_DEVICES="0"

(/home/thsch026/my-envs/PruneMe) 


### 4 Analyse für die Extraktion

In [61]:
cd '/home/thsch026/masterarbeit/experiment/PruneMe/compute_block_similarity'
python layer_similarity.py --model_path $MODEL \
                      --dataset "arcee-ai/sec-data-mini" \
                      --dataset_column "text" \
                      --batch_size 8 \
                      --max_length 1024 \
                      --layers_to_skip $LAYERSPRUNED \
                      --dataset_size 4000 \
                      --dataset_subset "train" 

(/home/thsch026/my-envs/PruneMe) 
/home/thsch026/my-envs/PruneMe/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|█████████████████| 2/2 [04:03<00:00, 121.78s/it]
/home/thsch026/my-envs/PruneMe/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing batches: 100%|█████████████████████| 500/500 [01

#### 5. Sichern der Resultats

In [62]:
mkdir $RESULTS
cp layer_distances.csv $RESULTS

(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 


### 5. Ausführen des Merging Tasks

In [99]:
export MERGE='/home/thsch026/masterarbeit/experiment/PruneMe/slice_with_mergekit'
cp $RESULTS/slice.yaml $MERGE
cd $MERGE
python merge_me.py

(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 
(/home/thsch026/my-envs/PruneMe) 
/home/thsch026/my-envs/PruneMe/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Executing graph: 100%|████████████████████████| 734/734 [03:44<00:00,  3.27it/s]
/home/thsch026/my-envs/PruneMe/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Done!
(/home/thsch026/my-envs/PruneMe) 


In [100]:
mv merged $PRUNEMODEL

(/home/thsch026/my-envs/PruneMe) 


In [101]:
cd $PRUNEMODEL
pwd
ls -ltr


(/home/thsch026/my-envs/PruneMe) 
/home/thsch026/masterarbeit/models/generated/prune/pruneme/Meta-Llama-3-8B-12_Ext
(/home/thsch026/my-envs/PruneMe) 
total 8411081
-rw-r--r-- 1 jovyan users 4953586384 Jun 30 19:23 model-00001-of-00003.safetensors
-rw-r--r-- 1 jovyan users 4999819296 Jun 30 19:24 model-00002-of-00003.safetensors
-rw-r--r-- 1 jovyan users        762 Jun 30 19:25 README.md
-rw-r--r-- 1 jovyan users      14314 Jun 30 19:25 model.safetensors.index.json
-rw-r--r-- 1 jovyan users  872450064 Jun 30 19:25 model-00003-of-00003.safetensors
-rw-r--r-- 1 jovyan users        198 Jun 30 19:25 mergekit_config.yml
-rw-r--r-- 1 jovyan users        698 Jun 30 19:25 config.json
-rw-r--r-- 1 jovyan users    9085671 Jun 30 19:25 tokenizer.json
-rw-r--r-- 1 jovyan users      50566 Jun 30 19:25 tokenizer_config.json
-rw-r--r-- 1 jovyan users        301 Jun 30 19:25 special_tokens_map.json
(/home/thsch026/my-envs/PruneMe) 
